In [6]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [7]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [8]:
!wget $data -O course_lead_scoring.csv

--2025-10-14 01:37:57--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.03s   

2025-10-14 01:37:58 (2.76 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [9]:
df = pd.read_csv('course_lead_scoring.csv')

In [10]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [11]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [12]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)
numerical = ['annual_income', 'interaction_count', 'lead_score', 'number_of_courses_viewed']

In [13]:
for col in df.columns:
    if col == 'converted':
        continue
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('NA')
    else:
        df[col] = df[col].fillna(0.0)

df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [14]:
#Q1

df.industry.mode()[0]

'retail'

In [15]:
# Q2
df[numerical].corrwith(df.converted)

annual_income               0.053131
interaction_count           0.374573
lead_score                  0.193673
number_of_courses_viewed    0.435914
dtype: float64

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [18]:
y_train = df_train.converted.values
y_test = df_test.converted.values
y_val = df_val.converted.values

In [19]:
del df_train['converted']
del df_test['converted']
del df_val['converted']

In [20]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

In [21]:
# Q3

from sklearn.metrics import mutual_info_score

def mutual_info_converted_score(series):
    return mutual_info_score(df_full_train.converted, series)

In [22]:
mi = df_full_train[categorical].apply(mutual_info_converted_score)
mi.sort_values(ascending=False).round(2)

lead_source          0.03
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

In [42]:
# Q4 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

def train_and_test_logistic_regression(df_train, y_train, df_val, y_val):
    train_dicts = df_train.to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dicts)
    
    X_train = dv.transform(train_dicts)
    
    val_dicts = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    converted_decision = (y_pred >= 0.5)
    
    return (y_val == converted_decision).mean()

In [53]:
init_accuracy = train_and_test_logistic_regression(df_train, y_train, df_val, y_val)
init_accuracy

np.float64(0.6996587030716723)

In [54]:
round(train_and_test_logistic_regression(df_train, y_train, df_val, y_val), 2)

np.float64(0.7)

In [ ]:
# Q5

def feature_elimination(df_train, y_train, df_val, y_val):
    columns = df_train.columns
    result_when_col_excluded = {}

    for col in columns:
        df_train_ex = df_train.loc[:, df_train.columns != col]
        df_val_ex = df_val.loc[:, df_val.columns != col]
        result_when_col_excluded[col] = train_and_test_logistic_regression(df_train_ex, y_train, df_val_ex, y_val)

    return result_when_col_excluded
    
result = feature_elimination(df_train, y_train, df_val, y_val)
result

In [58]:
result_df = pd.DataFrame.from_dict(result, orient='index')
result_df['diff'] = init_accuracy - result_df[0] 
result_df

,0,diff
lead_source,0.703072,-0.003413
industry,0.699659,0.000000
number_of_courses_viewed,0.556314,0.143345
annual_income,0.853242,-0.153584
employment_status,0.696246,0.003413
location,0.709898,-0.010239
interaction_count,0.556314,0.143345
lead_score,0.706485,-0.006826


In [153]:
# Q6 

def train_log_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    
    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])
    
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)

    return w_full[0], w_full[1:]
    pass


def logistic_regression_reg(xi):
    score = w0

    # print("len xi", len(xi))

    for j in range(len(w)):
        score = score + xi[j] * w[j]
        # print("xi[j]", xi[j])
        # print("score", score)
    
    result = sigmoid(score)
    # print(result)
    return result

def sigmoid(z):
    return 1 / (1+np.exp(-z))

def rmse(y, y_pred):
    error = y - y_pred
    se = error ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [214]:
def parameter_tuning(df_train, y_train, df_val, y_val,r):
    
    train_dicts = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    dv.fit(train_dicts)
    
    X_train = dv.transform(train_dicts)
    
    w0, w = train_log_regression_reg(X_train, y_train, r=r)

    print(w0)

    val_dicts = df_val.to_dict(orient='records')
    X_val = dv.transform(val_dicts)
    
    y_pred = []
    
    for i in range(len(X_val)):
        y_pred.insert(i,logistic_regression_reg(X_val[i]))

    conv_decision = (np.array(y_pred) >= 0.5).astype(int)

    return (y_val == conv_decision).mean() 

In [215]:
for c in [0.01, 0.1, 1, 10, 100]:
    print("C {} -> {}".format(c, parameter_tuning(df_train, y_train, df_val, y_val, c)))

-0.10846618656489909
C 0.01 -> 0.6040955631399317
-0.10822496170747098
C 0.1 -> 0.6040955631399317
-0.10585793987829502
C 1 -> 0.6040955631399317
-0.08599412474602819
C 10 -> 0.6040955631399317
-0.01857417003049303
C 100 -> 0.6040955631399317
